In [1]:
import sys
sys.path.append("./src")
import os
import pandas as pd 
import numpy as np
import json
import glob
import shutil
import matplotlib.pyplot as plt
import PIL
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import xception

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2



from model_trans import *


#from tensorflow.keras.applications.xception import preprocess_input


In [ ]:
model = build_model("./pre-trained/vgg_face_weights.h5", 9)

In [2]:
model.summary()

NameError: name 'model' is not defined

In [2]:
json_path = "./config/age_parameters.json"

with open(json_path) as param:
    data = json.load(param)
    model_param = data["model_param"]
    data_info = data["load_data"]
param.close()

lr, epochs, batch_size, mapping_path, save_path, log_path, _ = model_param.values()
train_label_path, train_image_path, valid_label_path, valid_image_path, target, size = data_info.values()

In [3]:
pd.read_csv(train_label_path)["gender"].nunique()

2

In [50]:
def create_generator(csv_path, image_path, target, size, batch_size, preprocess_input, is_training):
    
    if is_training:
        horizontal_flip = False
        vertical_flip = False
    else:
        horizontal_flip = True
        vertical_flip = True
    
    df = pd.read_csv(csv_path)
    df["file"] = df["file"].apply(lambda x: os.path.join(image_path, x.split("/")[1]))
    
    imgdatagen = ImageDataGenerator(
        preprocessing_function = xception.preprocess_input,
        horizontal_flip = horizontal_flip, 
        vertical_flip = vertical_flip,
    )
    
    data_generator = imgdatagen.flow_from_dataframe(
        dataframe = df,
        directory = None,
        x_col = "file",
        y_col = target,
        target_size = (size, size),
        batch_size = batch_size,
        save_format = "jpg",
        shuffle = True
    )
    
    with open(mapping_path, "w") as f:
        json.dump(data_generator.class_indices, f)
    f.close()
    
    return data_generator

In [51]:
val = create_generator(valid_label_path, valid_image_path, 
                       target, size, batch_size, densenet.preprocess_input, False)

Found 10954 validated image filenames belonging to 7 classes.


In [52]:
samp = next(iter(val))

In [32]:
samp[0]

array([[[[-0.40392154, -0.35686272, -0.35686272],
         [-0.40392154, -0.35686272, -0.35686272],
         [-0.40392154, -0.35686272, -0.35686272],
         ...,
         [-0.7882353 , -0.81960785, -0.8117647 ],
         [-0.78039217, -0.8117647 , -0.8039216 ],
         [-0.78039217, -0.8117647 , -0.8039216 ]],

        [[-0.40392154, -0.35686272, -0.35686272],
         [-0.40392154, -0.35686272, -0.35686272],
         [-0.40392154, -0.35686272, -0.35686272],
         ...,
         [-0.7882353 , -0.81960785, -0.8117647 ],
         [-0.7882353 , -0.81960785, -0.8117647 ],
         [-0.78039217, -0.8117647 , -0.8039216 ]],

        [[-0.40392154, -0.35686272, -0.35686272],
         [-0.40392154, -0.35686272, -0.35686272],
         [-0.40392154, -0.35686272, -0.35686272],
         ...,
         [-0.7882353 , -0.81960785, -0.8117647 ],
         [-0.7882353 , -0.81960785, -0.8117647 ],
         [-0.78039217, -0.8117647 , -0.8039216 ]],

        ...,

        [[-0.8745098 , -0.8901961 , -0

In [4]:
tr = ResNet50V2(include_top = False, 
                 weights = "imagenet",
                 input_shape = (224, 224, 3))

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

In [78]:
tr.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [89]:
for layer in tr.layers[:14]:
    print(layer.name)

input_7
conv1_pad
conv1_conv
pool1_pad
pool1_pool
conv2_block1_preact_bn
conv2_block1_preact_relu
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_pad
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu


In [79]:
len(tr.layers)

190

In [88]:
model = Sequential()

for layer in tr.layers[:16]:
    model.add(layer)

ValueError: Input 0 of layer conv2_block1_3_conv is incompatible with the layer: expected axis -1 of input shape to have value 64 but received input with shape [None, 56, 56, 256]

In [73]:
len(mod.layers)

192

In [ ]:
model = OutputModel().make_default_hidden_layers(size)

In [ ]:
size

In [ ]:
from tensorflow.keras import Sequential

tr = ResNet101V2(include_top = False, 
                          weights = "imagenet",
                          input_shape = (200, 200, 3))

In [ ]:
tr.summary()

In [ ]:
model = Sequential()

for layer in tr.layers[:15]:
            layer.trainable = False
            model.add(layer)
            print(layer.name)

In [ ]:
samp